In [10]:
import pandas as pd
data = pd.read_json('cladder.json')
data = data.dropna().reset_index(drop=True)

#create column "formal_form" and "graph" that holds the value corresponding to the key "formal_form" at the dict inside column "meta"

data = data.assign(formal_form = lambda x: (x['meta']))
df = data['formal_form'].apply(lambda x: (x.get("formal_form")))
data['formal_form'] = df

data = data.assign(graph = lambda x: (x['reasoning']))
df = data['graph'].apply(lambda x: (x.get("step1")))
data['graph'] = df

data = data.assign(info = lambda x: (x['meta']))
df = data['info'].apply(lambda x: (x.get("given_info")))
data['info'] = df
data


,question_id,desc_id,given_info,question,answer,meta,reasoning,formal_form,graph,info
0,4,alarm-mediation-nde-model0-spec0-q0,"For husbands that don't set the alarm and wives that don't set the alarm, the probability of ringing alarm is 8%. For husbands that don't set the alarm and wives that set the alarm, the probability of ringing alarm is 54%. For husbands that set the alarm and wives that don't set the alarm, the probability of ringing alarm is 41%. For husbands that set the alarm and wives that set the alarm, the probability of ringing alarm is 86%. For husbands that don't set the alarm, the probability of alarm set by wife is 74%. For husbands that set the alarm, the probability of alarm set by wife is 24%.","If we disregard the mediation effect through wife, would husband positively affect alarm clock?",yes,"{'story_id': 'alarm', 'graph_id': 'mediation', 'mediators': ['V2'], 'polarity': True, 'groundtruth': 0.322381664440195, 'query_type': 'nde', 'rung': 3, 'formal_form': 'E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]', 'given_info': {'p(Y | X, V2)': [[0.08430222457648501, 0.539461052145868], [0.406150970112692, 0.8620283206949241]], 'p(V2 | X)': [0.741686618881911, 0.23519324071521203]}, 'estimand': '\sum_{V2=v} P(V2=v|X=0)*[P(Y=1|X=1,V2=v) - P(Y=1|X=0, V2=v)]', 'treatment': 'X', 'outcome': 'Y', 'model_id': 0}","{'step0': 'Let X = husband; V2 = wife; Y = alarm clock.', 'step1': 'X->V2,X->Y,V2->Y', 'step2': 'E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]', 'step3': '\sum_{V2=v} P(V2=v|X=0)*[P(Y=1|X=1,V2=v) - P(Y=1|X=0, V2=v)]', 'step4': 'P(Y=1 | X=0, V2=0) = 0.08 P(Y=1 | X=0, V2=1) = 0.54 P(Y=1 | X=1, V2=0) = 0.41 P(Y=1 | X=1, V2=1) = 0.86 P(V2=1 | X=0) = 0.74 P(V2=1 | X=1) = 0.24', 'step5': '0.74 * (0.86 - 0.41) + 0.24 * (0.54 - 0.08) = 0.32', 'end': '0.32 > 0'}","E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]","X->V2,X->Y,V2->Y","{'p(Y | X, V2)': [[0.08430222457648501, 0.539461052145868], [0.406150970112692, 0.8620283206949241]], 'p(V2 | X)': [0.741686618881911, 0.23519324071521203]}"
1,7,alarm-mediation-ate-model1-spec1-q1,"For husbands that don't set the alarm, the probability of ringing alarm is 26%. For husbands that set the alarm, the probability of ringing alarm is 76%.",Will alarm set by husband decrease the chance of ringing alarm?,no,"{'story_id': 'alarm', 'graph_id': 'mediation', 'treated': True, 'result': True, 'polarity': False, 'groundtruth': 0.49506904015332404, 'query_type': 'ate', 'rung': 2, 'formal_form': 'E[Y | do(X = 1)] - E[Y | do(X = 0)]', 'given_info': {'p(Y | X)': [0.26076060984306904, 0.755829649996393]}, 'estimand': 'P(Y=1|X=1) - P(Y=1|X=0)', 'treatment': 'X', 'outcome': 'Y', 'model_id': 1}","{'step0': 'Let X = husband; V2 = wife; Y = alarm clock.', 'step1': 'X->V2,X->Y,V2->Y', 'step2': 'E[Y | do(X = 1)] - E[Y | do(X = 0)]', 'step3': 'P(Y=1|X=1) - P(Y=1|X=0)', 'step4': 'P(Y=1 | X=0) = 0.26 P(Y=1 | X=1) = 0.76', 'step5': '0.76 - 0.26 = 0.50', 'end': '0.50 > 0'}",E[Y | do(X = 1)] - E[Y | do(X = 0)],"X->V2,X->Y,V2->Y","{'p(Y | X)': [0.26076060984306904, 0.755829649996393]}"
2,8,alarm-mediation-marginal-model1-spec1-q0,"The overall probability of alarm set by husband is 77%. For husbands that don't set the alarm, the probability of ringing alarm is 26%. For husbands that set the alarm, the probability of ringing alarm is 76%.",Is ringing alarm more likely than silent alarm overall?,yes,"{'story_id': 'alarm', 'graph_id': 'mediation', 'given_info': {'P(X)': 0.771884206668314, 'P(Y | X)': [0.26076060984306904, 0.755829649996393]}, 'treated': True, 'polarity': True, 'groundtruth': 0.6429672171752431, 'query_type': 'marginal', 'rung': 1, 'formal_form': 'P(Y)', 'treatment': 'X', 'outcome': 'Y', 'model_id': 1}","{'step0': 'Let X = husband; V2 = wife; Y = alarm clock.', 'step1': 'X->V2,X->Y,V2->Y', 'step2': 'P(Y)', 'step3': 'P(Y | X=1)*P(X=1) + P(Y | X=0)*P(X=0)', 'step4': 'P(X=1) = 0.77 P(Y=1 | X=0) = 0.26 P(Y=1 | X=1) = 0.76', 'step5': '0.77*0.76 - 0.23*0.26 = 0.64', 'end': '0.64 > 0'}",P(Y),"X->V2,X->Y,V2->Y","{'P(X)': 0.771884206668314, 'P(

In [16]:
#exchange "formal_form" and "graph" column's variables to their respective "true name" (ex: exchange X to husband, V2 to wife, etc)
def exchange(x, form, aux):
    x = x.get('step0')
    if(aux == 1):
        form = form.get('formal_form')
    elif(aux == 0):
        form = form.get('step1')
    else:
        form = str(form)
    
    x = x[4:len(x)-1] #erase "Let " and the last "."
    x = x.split('; ')

    for i in range(len(x)):
        aux = x[i].split(" = ")
        form = form.replace(aux[0],aux[1])
    return form

data['formal_form'] = data.apply(lambda x: exchange(x['reasoning'], x['meta'],1), axis=1)
data['graph'] = data.apply(lambda x: exchange(x['reasoning'], x['reasoning'],0), axis=1)
data['info'] = data.apply(lambda x: exchange(x['reasoning'], x['info'],2), axis=1)


data

,question_id,desc_id,given_info,question,answer,meta,reasoning,formal_form,graph,info
0,4,alarm-mediation-nde-model0-spec0-q0,"For husbands that don't set the alarm and wives that don't set the alarm, the probability of ringing alarm is 8%. For husbands that don't set the alarm and wives that set the alarm, the probability of ringing alarm is 54%. For husbands that set the alarm and wives that don't set the alarm, the probability of ringing alarm is 41%. For husbands that set the alarm and wives that set the alarm, the probability of ringing alarm is 86%. For husbands that don't set the alarm, the probability of alarm set by wife is 74%. For husbands that set the alarm, the probability of alarm set by wife is 24%.","If we disregard the mediation effect through wife, would husband positively affect alarm clock?",yes,"{'story_id': 'alarm', 'graph_id': 'mediation', 'mediators': ['V2'], 'polarity': True, 'groundtruth': 0.322381664440195, 'query_type': 'nde', 'rung': 3, 'formal_form': 'E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]', 'given_info': {'p(Y | X, V2)': [[0.08430222457648501, 0.539461052145868], [0.406150970112692, 0.8620283206949241]], 'p(V2 | X)': [0.741686618881911, 0.23519324071521203]}, 'estimand': '\sum_{V2=v} P(V2=v|X=0)*[P(Y=1|X=1,V2=v) - P(Y=1|X=0, V2=v)]', 'treatment': 'X', 'outcome': 'Y', 'model_id': 0}","{'step0': 'Let X = husband; V2 = wife; Y = alarm clock.', 'step1': 'X->V2,X->Y,V2->Y', 'step2': 'E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]', 'step3': '\sum_{V2=v} P(V2=v|X=0)*[P(Y=1|X=1,V2=v) - P(Y=1|X=0, V2=v)]', 'step4': 'P(Y=1 | X=0, V2=0) = 0.08 P(Y=1 | X=0, V2=1) = 0.54 P(Y=1 | X=1, V2=0) = 0.41 P(Y=1 | X=1, V2=1) = 0.86 P(V2=1 | X=0) = 0.74 P(V2=1 | X=1) = 0.24', 'step5': '0.74 * (0.86 - 0.41) + 0.24 * (0.54 - 0.08) = 0.32', 'end': '0.32 > 0'}","E[alarm clock_{husband=1, wife=0} - alarm clock_{husband=0, wife=0}]","husband->wife,husband->alarm clock,wife->alarm clock","{'p(alarm clock | husband, wife)': [[0.08430222457648501, 0.539461052145868], [0.406150970112692, 0.8620283206949241]], 'p(wife | husband)': [0.741686618881911, 0.23519324071521203]}"
1,7,alarm-mediation-ate-model1-spec1-q1,"For husbands that don't set the alarm, the probability of ringing alarm is 26%. For husbands that set the alarm, the probability of ringing alarm is 76%.",Will alarm set by husband decrease the chance of ringing alarm?,no,"{'story_id': 'alarm', 'graph_id': 'mediation', 'treated': True, 'result': True, 'polarity': False, 'groundtruth': 0.49506904015332404, 'query_type': 'ate', 'rung': 2, 'formal_form': 'E[Y | do(X = 1)] - E[Y | do(X = 0)]', 'given_info': {'p(Y | X)': [0.26076060984306904, 0.755829649996393]}, 'estimand': 'P(Y=1|X=1) - P(Y=1|X=0)', 'treatment': 'X', 'outcome': 'Y', 'model_id': 1}","{'step0': 'Let X = husband; V2 = wife; Y = alarm clock.', 'step1': 'X->V2,X->Y,V2->Y', 'step2': 'E[Y | do(X = 1)] - E[Y | do(X = 0)]', 'step3': 'P(Y=1|X=1) - P(Y=1|X=0)', 'step4': 'P(Y=1 | X=0) = 0.26 P(Y=1 | X=1) = 0.76', 'step5': '0.76 - 0.26 = 0.50', 'end': '0.50 > 0'}",E[alarm clock | do(husband = 1)] - E[alarm clock | do(husband = 0)],"husband->wife,husband->alarm clock,wife->alarm clock","{'p(alarm clock | husband)': [0.26076060984306904, 0.755829649996393]}"
2,8,alarm-mediation-marginal-model1-spec1-q0,"The overall probability of alarm set by husband is 77%. For husbands that don't set the alarm, the probability of ringing alarm is 26%. For husbands that set the alarm, the probability of ringing alarm is 76%.",Is ringing alarm more likely than silent alarm overall?,yes,"{'story_id': 'alarm', 'graph_id': 'mediation', 'given_info': {'P(X)': 0.771884206668314, 'P(Y | X)': [0.26076060984306904, 0.755829649996393]}, 'treated': True, 'polarity': True, 'groundtruth': 0.6429672171752431, 'query_type': 'marginal', 'rung': 1, 'formal_form': 'P(Y)', 'treatment': 'X', 'outcome': 'Y', 'model_id': 1}","{'step0': 'Let X = husband; V2 = wife; Y = alarm clock.', 'step1': 'X->V2,X->Y,V2->Y', 'step2': 'P(Y)', 'step3': 'P(Y | X=1)*P(X=1) + P(Y | X=0)*P(X=0)', '

In [3]:
pd.set_option('display.max_colwidth', None)

In [17]:
#create columnn with complete prompt to be given to the LLM

prompt_start = "Question: "
prompt_end = "\nGuidance: Address the question above by following the steps below:\nStep 1) Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. The diagram should simply consist of edges denoted in 'var1 -> var2' format, separated by commas. Step 2) Determine the query type: Identify the type of query implied by the main question. Choices include 'marginal probability', 'conditional probability', 'explaining away effect', 'backdoor adjustment set', 'average treatment effect', 'collider bias', 'normal counterfactual question', 'average treatment effect on treated', 'natural direct effect' or 'natural indirect effect'. Your answer should only be a term from the list above, enclosed in quotation marks. Step 3) Formalize the query: Translate the query into its formal mathematical expression based on its type, utilizing the 'do(·)' notation or counterfactual notations as needed."

data = data.assign(prompt = lambda x: (prompt_start+x['given_info']+x['question']+prompt_end))
df = data[['prompt', 'formal_form', 'graph', 'info']]
df

,prompt,formal_form,graph,info
0,"Question: For husbands that don't set the alarm and wives that don't set the alarm, the probability of ringing alarm is 8%. For husbands that don't set the alarm and wives that set the alarm, the probability of ringing alarm is 54%. For husbands that set the alarm and wives that don't set the alarm, the probability of ringing alarm is 41%. For husbands that set the alarm and wives that set the alarm, the probability of ringing alarm is 86%. For husbands that don't set the alarm, the probability of alarm set by wife is 74%. For husbands that set the alarm, the probability of alarm set by wife is 24%.If we disregard the mediation effect through wife, would husband positively affect alarm clock?\nGuidance: Address the question above by following the steps below:\nStep 1) Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. The diagram should simply consist of edges denoted in 'var1 -> var2' format, separated by commas. Step 2) Determine the query type: Identify the type of query implied by the main question. Choices include 'marginal probability', 'conditional probability', 'explaining away effect', 'backdoor adjustment set', 'average treatment effect', 'collider bias', 'normal counterfactual question', 'average treatment effect on treated', 'natural direct effect' or 'natural indirect effect'. Your answer should only be a term from the list above, enclosed in quotation marks. Step 3) Formalize the query: Translate the query into its formal mathematical expression based on its type, utilizing the 'do(·)' notation or counterfactual notations as needed.","E[alarm clock_{husband=1, wife=0} - alarm clock_{husband=0, wife=0}]","husband->wife,husband->alarm clock,wife->alarm clock","{'p(alarm clock | husband, wife)': [[0.08430222457648501, 0.539461052145868], [0.406150970112692, 0.8620283206949241]], 'p(wife | husband)': [0.741686618881911, 0.23519324071521203]}"
1,"Question: For husbands that don't set the alarm, the probability of ringing alarm is 26%. For husbands that set the alarm, the probability of ringing alarm is 76%.Will alarm set by husband decrease the chance of ringing alarm?\nGuidance: Address the question above by following the steps below:\nStep 1) Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. The diagram should simply consist of edges denoted in 'var1 -> var2' format, separated by commas. Step 2) Determine the query type: Identify the type of query implied by the main question. Choices include 'marginal probability', 'conditional probability', 'explaining away effect', 'backdoor adjustment set', 'average treatment effect', 'collider bias', 'normal counterfactual question', 'average treatment effect on treated', 'natural direct effect' or 'natural indirect effect'. Your answer should only be a term from the list above, enclosed in quotation marks. Step 3) Formalize the query: Translate the query into its formal mathematical expression based on its type, utilizing the 'do(·)' notation or counterfactual notations as needed.",E[alarm clock | do(husband = 1)] - E[alarm clock | do(husband = 0)],"husband->wife,husband->alarm clock,wife->alarm clock","{'p(alarm clock | husband)': [0.26076060984306904, 0.755829649996393]}"
2,"Question: The overall probability of alarm set by husband is 77%. For husbands that don't set the alarm, the probability of ringing alarm is 26%. For husbands that set the alarm, the probability of ringing alarm is 76%.Is ringing alarm more likely than silent alarm overall?\nGuidance: Address the question above by following the steps below:\nStep 1) Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. The diagram should simply consist of edges denoted in 'var1 -> var2' format, separated by commas. Step 2) Determine the query type: Identify the type of query implied by the main question. Choices include 'marginal probability', '

In [18]:
import transformers

import torch

model_id = "meta-llama/Llama-2-7b-hf"

pipeline = transformers.pipeline(
    "text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto"
)

#pipeline("Hey how are you doing today?")

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [19]:
pipeline("Hey how are you doing today?")

OutOfMemoryError: CUDA out of memory. Tried to allocate 250.00 MiB. GPU 0 has a total capacity of 10.91 GiB of which 141.19 MiB is free. Including non-PyTorch memory, this process has 10.77 GiB memory in use. Of the allocated memory 10.44 GiB is allocated by PyTorch, and 169.47 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [13]:
pipeline.device

device(type='cuda', index=0)

In [14]:
round(0.23519324071521203,2)

0.24